In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [24]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, and ALL RANK COLUMNS and all columns with less than .3% feature
#importances in previous iteration
X = X.drop(columns=['UnitID', 'Rank', 'Name', 'Graduation rate rank', 'Graduation rate performance rank', 'Pell graduation gap rank', 
     'Pell performance rank',
     'Earnings performance rank',
     'Net price rank', 'Repayment rank', 'Repayment rate performance rank',
     'Research expenditures rank', "Bachelor's to PhD rank",
     'Science & engineering PhDs rank', 'Faculty accolades rank', 'AmeriCorps/Peace Corps rank', 'ROTC rank',
     '% of federal work-study funds spent on service rank',
     'Service-oriented majors rank', 'Social mobility rank', 'Research rank',
     'Service rank', 'Voting engagement points', 'Research expenditures, in millions', '% of federal work-study funds spent on service',
    '% of grads with service-oriented majors', 'Predicted median earnings 10 years after entering college',
    'Earns Carnegie Community Engagement Classification?', 'Science & engineering PhDs awarded',
    'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
    'Faculty in National Academies', 'Predicted principal remaining'])
X.head()

#TRY RETAINING ONLY THE RANKINGS?


,8-year graduation rate,Pell/non-Pell graduation gap,Number of Pell graduates,Actual vs. predicted Pell enrollment,Median earnings 10 years after entering college,"Net price of attendance for families below $75,000 income",% of loan principal remaining 5 years later,Faculty receiving significant awards
0,0.963302,-0.021703,260.6667,0.038096,97798,1445.8400,0.322153,0.044643
1,0.964294,-0.041917,376.6667,-0.000590,103246,7368.1700,0.351408,0.019748
2,0.950971,-0.032985,182.6667,0.076769,111222,3290.8450,0.305619,0.028059
3,0.976861,-0.052740,153.3333,0.062997,95689,3672.0020,0.360991,0.026616
4,0.954474,-0.014278,231.6667,0.028513,93115,671.8563,0.361023,0.014617


In [25]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 8 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   8-year graduation rate                                     442 non-null    float64
 1   Pell/non-Pell graduation gap                               442 non-null    float64
 2   Number of Pell graduates                                   442 non-null    float64
 3   Actual vs. predicted Pell enrollment                       442 non-null    float64
 4   Median earnings 10 years after entering college            442 non-null    int64  
 5   Net price of attendance for families below $75,000 income  442 non-null    float64
 6   % of loan principal remaining 5 years later                442 non-null    float64
 7   Faculty receiving significant awards                       442 non-null    float64
dtypes: float64(

In [26]:
# Define the target set.
y = df2_rankings["Rank"].ravel()
y[:5]


array([1, 2, 3, 4, 5], dtype=int64)

In [27]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [28]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 8)
Training Labels Shape: (331,)
Testing Features Shape: (111, 8)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [29]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [30]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [31]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)

The r2 score for this Random Forest Regressor model is: 0.7336103983380635


In [32]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 48.14
Accuracy: 59.66 %.


In [33]:
y_test

array([340,  46, 201, 374, 111, 355, 295, 169, 303,  84,  70,  68, 140,
       160, 222, 434, 215, 133,   7,  31, 276, 336, 263, 314,  99, 370,
       227, 273, 206, 126, 327, 250, 422, 284, 142, 192, 136, 436, 313,
       428, 168, 171, 337, 230, 204, 367, 157,  25, 292,  64, 262, 196,
         8, 254, 299,  75, 368, 361, 187,  22, 259,  93, 256,  16, 363,
       184, 249,  33,  20, 366,   3,  95, 210,  52, 182, 281, 128,  92,
       382, 349,  49, 411, 158, 224, 383, 421, 387, 351,  51, 195, 296,
        17, 274, 202,  88, 241,  21,  91, 424,  76, 137, 258,  42, 243,
       392, 309,  72, 401, 350, 233, 221], dtype=int64)

In [34]:
errors

array([8.71140e+01, 6.81480e+01, 1.32530e+01, 3.00500e+01, 6.92120e+01,
       1.45890e+01, 9.63100e+00, 7.01250e+01, 5.32420e+01, 2.39820e+01,
       4.10590e+01, 8.71500e+01, 1.28660e+02, 2.55080e+01, 1.17740e+01,
       9.38210e+01, 8.70950e+01, 6.56500e+00, 2.29500e+00, 1.27310e+01,
       1.64950e+01, 3.00630e+01, 2.51520e+01, 6.79770e+01, 2.90480e+01,
       9.47500e+01, 4.47420e+01, 1.43170e+01, 5.90210e+01, 5.63010e+01,
       2.13240e+01, 1.89680e+01, 2.37690e+01, 8.50110e+01, 1.01295e+02,
       1.15791e+02, 2.83540e+01, 7.92090e+01, 5.70530e+01, 4.51080e+01,
       5.13150e+01, 3.84890e+01, 3.99700e+01, 4.23600e+00, 2.50630e+01,
       1.20582e+02, 5.55480e+01, 1.51990e+01, 3.19450e+01, 1.66004e+02,
       1.38575e+02, 3.67790e+01, 1.01460e+01, 1.12170e+01, 2.29830e+01,
       4.02780e+01, 6.66770e+01, 2.80813e+02, 1.02039e+02, 1.15784e+02,
       1.86850e+01, 2.52500e+01, 1.72460e+01, 2.55380e+01, 3.64280e+01,
       1.10980e+01, 8.89230e+01, 2.65070e+01, 1.03010e+01, 2.095

In [35]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 48.13762162162161
Mean Squared Error: 4143.261319243244
Root Mean Squared Error: 64.36817007841098


In [36]:
# Other ways of quantifying accuracy


# # Find the original feature indices 
# original_feature_indices = [feature_list.index(feature) for feature in feature_list if feature not in ['ws_1', 'prcp_1', 'snwd_1']]
# # Create a test set of the original features
# original_test_features = test_features[:, original_feature_indices]
# # Make predictions on test data using the model trained on original data
# predictions = rf.predict(original_test_features)
# # Performance metrics
# errors = abs(predictions - test_labels)
# print('Metrics for Random Forest Trained on Original Data')
# print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')


In [37]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.19660469, 0.09457081, 0.04310673, 0.05084525, 0.05283517,
       0.17063074, 0.08640965, 0.30499696])

In [38]:
# #plotting feature importances

# # Get numerical feature importances
# importances = list(rf_exp.feature_importances_)
# # List of tuples with variable and importance
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# # Sort the feature importances by most important first
# feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# # Print out the feature and importances 
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# # list of x locations for plotting
# x_values = list(range(len(importances)))
# # Make a bar chart
# plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# # Tick labels for x axis
# plt.xticks(x_values, feature_list, rotation='vertical')
# # Axis labels and title
# plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [39]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.304997,Faculty receiving significant awards
1,0.196605,8-year graduation rate
2,0.170631,Net price of attendance for families below $75...
3,0.094571,Pell/non-Pell graduation gap
4,0.086410,% of loan principal remaining 5 years later
5,0.052835,Median earnings 10 years after entering college
6,0.050845,Actual vs. predicted Pell enrollment
7,0.043107,Number of Pell graduates


In [40]:
sorted_df.to_csv('Feature_Importances_with_all_RANK_and_LEAST_IMPORTANT_features_removed.csv')  

## Support Vector Regression

In [41]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

## the kernel is the most important part - try different kernels

# Fit the data

regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model


SVR(kernel='poly')

In [42]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [43]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)


The r2 score for this SVM model is: 0.3269291262640942


In [44]:
#visualize SVR results: https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/


## Decision Tree Regression

In [45]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [46]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [47]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.5459927750767513


## Multiple Linear Regression

In [48]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [49]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [50]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.7356161817817657
